In [92]:
import os
from glob import glob
from pathlib import Path
import shutil
import sys
import time
from io import BytesIO
from tqdm import tqdm

In [73]:
today = time.strftime('%Y-%m-%d')

In [74]:
def transfer_file(source, dest_path):
    fsize = int(os.path.getsize(source))
    dest = os.path.join(dest_path, os.path.basename(source))
    with open(source, 'rb') as f:
        with open(dest, 'ab') as n:
            with tqdm(ncols=60, total=fsize, bar_format='{l_bar}{bar} | Remaining: {remaining}') as pbar:
                buffer = bytearray()
                while True:
                    buf = f.read(8192)
                    n.write(buf)
                    if len(buf) == 0:
                        break
                    buffer += buf
                    pbar.update(len(buf))

In [75]:
def get_all_files(card_path):
    all_files = []
    for root, dirs, files in os.walk(card_path):
        for file in files:
            if '.Trashes' not in root:
                all_files.append(os.path.join(root, file))
    return all_files

In [76]:
cards = glob('/media/landry/*')
cards

['/media/landry/3461-3834',
 '/media/landry/3239-3631',
 '/media/landry/6261-6434',
 '/media/landry/6137-3234']

In [77]:
get_all_files(cards[0])

['/media/landry/3461-3834/mdb_max.db',
 '/media/landry/3461-3834/Get_started_with_GoPro.url',
 '/media/landry/3461-3834/mdb_max.bk',
 '/media/landry/3461-3834/mdb_max.log',
 '/media/landry/3461-3834/DCIM/leinfo.sav',
 '/media/landry/3461-3834/DCIM/100GOPRO/GS010008.360',
 '/media/landry/3461-3834/DCIM/100GOPRO/GS020008.360',
 '/media/landry/3461-3834/DCIM/100GOPRO/GS030008.360',
 '/media/landry/3461-3834/MISC/version.txt',
 '/media/landry/3461-3834/MISC/sd_info.txt',
 '/media/landry/3461-3834/MISC/card']

In [78]:
# get paths of SD cards connected
if sys.platform == 'darwin':
    connected_sd_cards = [Path(card) for card in glob('/Volumes/Untitled*/')]
elif sys.platform == 'win32':
    connected_sd_cards = [Path(card) for card in glob('E:/')]
elif sys.platform == 'linux':
    user = os.environ['USER']
    connected_sd_cards = [Path(card) for card in glob(f'/media/{user}/*')]
else:
    # throw error if platform is unknown
    raise OSError('Unknown platform')

In [79]:
card_id = {}
# card label and files to download
data_incoming = {''}
cam_id = 0
for card in connected_sd_cards:
    if any(['.360' in file for file in get_all_files(card)]):
        files = [file for file in get_all_files(card) if '.360' in file]
        card_id['360cam'] = {'card': str(card), 'files': files}
    elif any(['.mp4' in file.lower() for file in get_all_files(card)]):
        cam_id+=1
        files = [file for file in get_all_files(card) if '.mp4' in file.lower()]
        card_id[f'cam{cam_id}'] = {'card': str(card), 'files': files}
    elif any(['TRACK' in file for file in get_all_files(card)]):
        # TO DO: make sure audio files recorded on same date as video files
        files = [file for file in get_all_files(str(card)) if 'TRACK' in file]
        card_id['microphones'] = {'card': card, 'files': files}
    else:
        print(f'Unknown card: {card} contains the following files:')
        print(get_all_files(card))

In [80]:
card_id

{'360cam': {'card': '/media/landry/3461-3834',
  'files': ['/media/landry/3461-3834/DCIM/100GOPRO/GS010008.360',
   '/media/landry/3461-3834/DCIM/100GOPRO/GS020008.360',
   '/media/landry/3461-3834/DCIM/100GOPRO/GS030008.360']},
 'microphones': {'card': PosixPath('/media/landry/3239-3631'),
  'files': ['/media/landry/3239-3631/test2/TRACK01.WAV',
   '/media/landry/3239-3631/test2/TRACK02.WAV',
   '/media/landry/3239-3631/test2/TRACK03.WAV',
   '/media/landry/3239-3631/test2/TRACK04.WAV']},
 'cam1': {'card': '/media/landry/6261-6434',
  'files': ['/media/landry/6261-6434/DCIM/100GOPRO/GH010020.mp4',
   '/media/landry/6261-6434/DCIM/100GOPRO/GH020020.mp4']},
 'cam2': {'card': '/media/landry/6137-3234',
  'files': ['/media/landry/6137-3234/DCIM/100GOPRO/GH010020.MP4',
   '/media/landry/6137-3234/DCIM/100GOPRO/GH020020.MP4']}}

In [81]:
assert len(card_id) == 4, 'Not all cards are connected'

In [82]:
# func for getting the creation date of a file
def get_creation_date(path_to_file):
    return time.strftime('%Y-%m-%d', time.gmtime(os.path.getctime(path_to_file)))

In [83]:
# make sure all files have the same creation date and print the files for each unique date detected
for card in card_id:
    dates = []
    for file in card_id[card]['files']:
        dates.append(get_creation_date(file))
    dates = list(set(dates))
    assert len(dates) == 1, f'Files on {card} were not all created on the same date'
    print(f'Files on {card} were created on {dates[0]}')

Files on 360cam were created on 2023-09-15
Files on microphones were created on 2023-09-15
Files on cam1 were created on 2023-09-15
Files on cam2 were created on 2016-03-02


In [89]:
# ask user if ok to transfer the files
dialog = 'The following files will be transferred:\n\n'
for card in card_id:
    dialog += f'{card} has {len(card_id[card]["files"])} files:\n'
    for file in card_id[card]['files']:
        dialog += f'\t{file}\n'
    dialog += '\n'
dialog += 'Is this ok? (y/n): '
ok = input(dialog)
if ok.lower() == 'y':
    transfer_approved = True
    pass
else:
    transfer_approved = False
    raise OSError('User aborted transfer')

In [90]:
def get_exp_of_day():
    # get all experiments of the day
    today = time.strftime('%Y-%m-%d')
    data_path = Path('/safestore/users/landry/SCRAP/data')
    exp_of_day = [exp for exp in glob(f'{data_path}/{today}*') if os.path.isdir(exp)]
    # get the number of the last experiment of the day
    if len(exp_of_day) == 0:
        exp_num = 0
    else:
        exp_num = max([int(exp.split('_')[-1]) for exp in exp_of_day])
    return exp_num

In [ ]:
data_path = Path('/safestore/users/landry/SCRAP/data') / f'{today}_{get_exp_of_day()+1:03d}'
os.mkdir(data_path)
total_num_files = sum([len(card_id[card]['files']) for card in card_id])
filenum = 0
for card in card_id:
    if card=='microphones':
        card = 'audio'
    dest_path = data_path / card
    os.mkdir(dest_path)
    for file in card_id[card]['files']:
        filenum += 1
        print('Transferring file {filenum} of {total_num_files} files\n')
        print('Destination: {dest_path}\n)
        transfer_file(file, dest_path)

Transferring file:  /media/landry/3461-3834/DCIM/100GOPRO/GS010008.360


100%|████████████████████████████████████ | Remaining: 00:00


Transferring file:  /media/landry/3461-3834/DCIM/100GOPRO/GS020008.360


100%|████████████████████████████████████ | Remaining: 00:00


Transferring file:  /media/landry/3461-3834/DCIM/100GOPRO/GS030008.360


...

100%|████████████████████████████████████ | Remaining: 00:00


Transferring file:  /media/landry/6261-6434/DCIM/100GOPRO/GH020020.mp4


100%|████████████████████████████████████ | Remaining: 00:00


Transferring file:  /media/landry/6137-3234/DCIM/100GOPRO/GH010020.MP4


 25%|████████▊                            | Remaining: 01:11